In [ ]:
!pip install sec_edgar_downloader==4.3.0

     |████████████████████████████████| 1.5 MB 14.5 MB/s 
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=3ea7e4ab0a09c18485035636b9c188f65ea68b66b570e468667cc5204ea1043b
  Stored in directory: /root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
!pip install openpyxl 


     |████████████████████████████████| 242 kB 4.4 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import numpy as np
import os
from sec_edgar_downloader import Downloader
import requests
import shutil
from bs4 import BeautifulSoup
import string
import random

In [ ]:
def files_downloader(file_type, comp_code, n):
    '''
    Function that takes as an input a company code (stock exchange code), a file type and a number of files to download
    Downloads all the corresponding files and stores them in the correct folder
    '''
    comp_path = './data/{comp}'.format(comp = comp_code)
    comp_raw_path = './data/{comp}/raw'.format(comp = comp_code)

    if not os.path.isdir(comp_path):
        os.makedirs(comp_path)
        
        if not os.path.isdir(comp_raw_path):
            os.makedirs(comp_raw_path)

    dl = Downloader(comp_raw_path)
    dl.get(file_type, comp_code, amount = n)

    # Get all the folders in the created folder
    files_path = os.path.join(comp_raw_path, 'sec-edgar-filings', comp_code, file_type)
    subf_list = [os.path.join(files_path, folder) for folder in os.listdir(files_path)]

    # Get all the final files
    final_files_list = [os.path.join(files_path, folder, 'full-submission.txt') for folder in os.listdir(files_path)]
    years_list = [folder.split('-')[1] for folder in os.listdir(files_path)]

    # Rename and move all the final files
    for i in range(len(final_files_list)):
        f_new_name = '{comp}_{year}.txt'.format(comp=comp_code, year=years_list[i])
        os.replace(final_files_list[i], './data/{comp}/raw/{f}'.format(comp=comp_code, f=f_new_name))
    
    # Delete the useless folders
    shutil.rmtree('./data/{comp}/raw/sec-edgar-filings'.format(comp=comp_code))


In [ ]:
# Download the years 2010-2022 10-k files from Apple
#files_downloader("10-K", "AAPL", n = 12)

In [ ]:
# Download the years 2010-2022 10-k files from Tesla
#files_downloader("10-K", "TSLA", n = 12)

In [ ]:
# Download the years 2010-2022 10-k files from Agilent
#files_downloader("10-K", "A", n = 12)

https://sec-edgar-downloader.readthedocs.io/en/latest/

https://www.sec.gov/os/accessing-edgar-data This is where i got json file with company codes (aka tickers)

NOTE: Converted JSON to Excel file using this website: https://www.convertcsv.com/json-to-csv.htm

In [ ]:
# Extract ticker/company code from Edgar SEC.Gov excel sheet (updated still in 2022)
# removed "-*" in excel before uploading to notebook. Include in write up
data = pd.read_excel (r'/work/company_tickers.xlsx', sheet_name='Sheet 2') #place "r" before the path string to address special character, such as '\'. Don't forget to put the file name at the end of the path + '.xlsx'
df = pd.DataFrame(data, columns= ['ticker'])
#print (df)


In [ ]:
pick20 = df.sample(n=20)
pick20


,ticker
1689,BOLT
3201,DMAQ
2748,CRLBF
6314,KOSCF
10540,TALK
8075,OIBZQ
10942,TRLFF
9152,RCACW
7274,MQ
10049,SMBK


In [ ]:
comp_codes = pick20.values.tolist()
#print(comp_codes)
flattened = []
for sublist in comp_codes:
    for val in sublist:
        flattened.append(val)

print(flattened)

['BOLT', 'DMAQ', 'CRLBF', 'KOSCF', 'TALK', 'OIBZQ', 'TRLFF', 'RCACW', 'MQ', 'SMBK', 'ESSI', 'SUND', 'AURI', 'KALV', 'GCC', 'ENPC', 'STCC', 'JP', 'DRCTW', 'WATT']


In [ ]:
for c in flattened:
    try:
        files_downloader("10-K", c, n=12)  # CHANGE HERE IF DESIRED 
    except(FileNotFoundError):
        print("Item Not Found %s" % c)
        continue
    

Item Not Found DMAQ
Item Not Found CRLBF
Item Not Found KOSCF
Item Not Found OIBZQ
Item Not Found TRLFF
Item Not Found AURI


In [ ]:
#files_downloader("10-K","ECF",n=2)

In [ ]:
def alltxt():
    directory = os.listdir('./data') #list of comp names
    allfiles = open("allfiles.txt", "a") #empty file to write

    table = str.maketrans('', '', string.punctuation)

    for comp_name in directory: #iterate through list of comp names
        if len(os.listdir("./data/{0}/raw".format(comp_name))) == 0:
            continue
        txt_file = random.choice(os.listdir("./data/{0}/raw".format(comp_name)))
        print('Start cleaning: ' + txt_file)
        current_file = open("./data/{0}/raw/{1}".format(comp_name,txt_file),"r")
        curr_text = current_file.read()
        current_file.close()
        curr_text = curr_text.lower()

        soup = BeautifulSoup(curr_text, "html.parser")
        lines = [x.get_text() for x in soup.find_all("p")]
        lines = [line.replace(u'\xa0', u' ').replace('\n', ' ') for line in lines]
        lines = [l.replace('☒','').replace('☐','').replace('•','').replace('“','').replace('”','').replace('’s','') for l in lines]
        lines = [l for l in lines if len(l.split()) > 1]
        
        stripped = [w.translate(table) for w in lines]

        allfiles.writelines([line + '\n' for line in stripped])
        print('Finished. Got ' + str(len(lines)) + ' lines.')


    allfiles.close()
#/work/data/EDUC/raw/{TEXTFILE}.txt.format(TEXTFILE = )


In [ ]:
alltxt()

Start cleaning: STCC_13.txt
Finished. Got 488 lines.
Start cleaning: ENPC_22.txt
Finished. Got 659 lines.
Start cleaning: KALV_16.txt
Finished. Got 1603 lines.
Start cleaning: GCC_08.txt
Finished. Got 137 lines.
Start cleaning: MQ_06.txt
Finished. Got 2288 lines.
Start cleaning: SMBK_16.txt
Finished. Got 935 lines.
Start cleaning: WATT_22.txt
Finished. Got 986 lines.
Start cleaning: DRCTW_22.txt
Finished. Got 939 lines.
Start cleaning: SUND_15.txt
Finished. Got 1444 lines.
Start cleaning: TALK_15.txt
Finished. Got 880 lines.
Start cleaning: ESSI_16.txt
Finished. Got 0 lines.
Start cleaning: RCACW_22.txt
Finished. Got 1781 lines.
Start cleaning: JP_15.txt
Finished. Got 2769 lines.
Start cleaning: BOLT_05.txt
Finished. Got 655 lines.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=62207f3c-7a44-4ad3-8df6-07126844dae3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>